# Assignment: Exploratory Data Analysis
**Student:** Vicky

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)

## Q1: Mathematical Properties

### 1.1: Show $m(a + bX) = a + b \times m(X)$

$$m(a + bX) = \frac{1}{N}\sum(a + bx_i) = \frac{1}{N}(Na + b\sum x_i) = a + b\cdot m(X)$$

### 1.2: Show $\text{cov}(X, a+bY) = b \times \text{cov}(X,Y)$

Since $m(a+bY) = a + b\cdot m(Y)$:
$$\text{cov}(X, a+bY) = \frac{1}{N}\sum(x_i - m(X))(a+by_i - a - b\cdot m(Y)) = b\cdot\text{cov}(X,Y)$$

### 1.3: Show $\text{cov}(a+bX, a+bX) = b^2\text{cov}(X,X)$ and $\text{cov}(X,X) = s^2$

Applying 1.2 twice: $\text{cov}(a+bX, a+bX) = b\cdot\text{cov}(X, a+bX) = b^2\cdot\text{cov}(X,X)$

$$\text{cov}(X,X) = \frac{1}{N}\sum(x_i - m(X))^2 = s^2$$

### 1.4: Non-decreasing transformations and median

**Yes**, for non-decreasing $g()$: $g(\text{median}(X)) = \text{median}(g(X))$

Proof: If 50% of values ≤ median($X$), then 50% of $g(x_i)$ ≤ $g($median($X$)$)$ since $g$ preserves order.

**Applies to all quantiles:** Yes - same logic applies to any percentile (IQR, range, etc.)

### 1.5: Is $m(g(X)) = g(m(X))$ for non-decreasing transformations?

**No.** Counterexample: $X = \{1,2,3\}$, $g(x) = x^2$
- $g(m(X)) = 2^2 = 4$
- $m(g(X)) = (1+4+9)/3 = 4.67$

Only true for linear transformations.

In [ ]:
# Verify counterexample
X = np.array([1, 2, 3])
print(f"g(m(X)) = {X.mean()**2:.2f}")
print(f"m(g(X)) = {(X**2).mean():.2f}")

## Q2: Foreign Gifts to US Universities

In [ ]:
# 2.1: Load data
gifts = pd.read_csv('/mnt/user-data/uploads/ForeignGifts_edu.csv')
print(f"Shape: {gifts.shape}")
gifts.head()

In [ ]:
# 2.2: Histogram and description of Foreign Gift Amount
print(gifts['Foreign Gift Amount'].describe())

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].hist(gifts['Foreign Gift Amount'], bins=50, edgecolor='black')
axes[0].set_title('Foreign Gift Amount')
axes[0].set_xlabel('Amount ($)')

axes[1].hist(gifts['Foreign Gift Amount'], bins=50, edgecolor='black')
axes[1].set_yscale('log')
axes[1].set_title('Foreign Gift Amount (Log Scale)')
axes[1].set_xlabel('Amount ($)')
plt.tight_layout()
plt.show()

print("\nFindings: Heavily right-skewed distribution with extreme outliers. Median << Mean.")

In [ ]:
# 2.3: Gift Type distribution
gift_counts = gifts['Gift Type'].value_counts()
gift_props = gifts['Gift Type'].value_counts(normalize=True) * 100

print("Gift Type Proportions:")
print(gift_props)

plt.figure(figsize=(10, 4))
gift_counts.plot(kind='bar', edgecolor='black')
plt.title('Distribution of Gift Types')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 2.4: KDE plots of log gift amount
gifts['log_amount'] = np.log(gifts['Foreign Gift Amount'] + 1)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

gifts['log_amount'].plot(kind='kde', ax=axes[0], linewidth=2)
axes[0].set_title('KDE of Log Gift Amount')
axes[0].set_xlabel('Log(Amount)')

for gift_type in gifts['Gift Type'].unique():
    gifts[gifts['Gift Type'] == gift_type]['log_amount'].plot(
        kind='kde', ax=axes[1], linewidth=2, label=gift_type)
axes[1].set_title('KDE by Gift Type')
axes[1].set_xlabel('Log(Amount)')
axes[1].legend()
plt.tight_layout()
plt.show()

print("\nPattern: Different gift types have distinct amount distributions.")

In [ ]:
# 2.5: Top 15 countries
top_count = gifts['Country of Giftor'].value_counts().head(15)
top_amount = gifts.groupby('Country of Giftor')['Foreign Gift Amount'].sum().sort_values(ascending=False).head(15)

fig, axes = plt.subplots(2, 1, figsize=(12, 8))

top_count.plot(kind='barh', ax=axes[0], edgecolor='black')
axes[0].set_title('Top 15 Countries by Number of Gifts')
axes[0].invert_yaxis()

top_amount.plot(kind='barh', ax=axes[1], color='coral', edgecolor='black')
axes[1].set_title('Top 15 Countries by Total Amount')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# 2.6: Top 15 institutions and histogram
top_inst = gifts.groupby('Institution Name')['Foreign Gift Amount'].sum().sort_values(ascending=False).head(15)
inst_totals = gifts.groupby('Institution Name')['Foreign Gift Amount'].sum()

fig, axes = plt.subplots(2, 1, figsize=(12, 8))

top_inst.plot(kind='barh', ax=axes[0], color='green', edgecolor='black')
axes[0].set_title('Top 15 Institutions by Total Amount')
axes[0].invert_yaxis()

axes[1].hist(inst_totals, bins=50, edgecolor='black', color='green')
axes[1].set_yscale('log')
axes[1].set_title('Distribution of Total Amounts (All Institutions)')
axes[1].set_xlabel('Total Amount ($)')

plt.tight_layout()
plt.show()

print(f"Top 15 capture {top_inst.sum() / gifts['Foreign Gift Amount'].sum() * 100:.1f}% of total gifts")

In [ ]:
# 2.7: Top giftors
top_giftors = gifts[gifts['Giftor Name'].notna()].groupby('Giftor Name')['Foreign Gift Amount'].sum().sort_values(ascending=False).head(15)

plt.figure(figsize=(12, 6))
top_giftors.plot(kind='barh', color='coral', edgecolor='black')
plt.title('Top 15 Giftors by Total Amount')
plt.xlabel('Total Amount ($)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Q3: Airbnb Data

In [ ]:
# 3.1 & 3.2: Load and examine
airbnb = pd.read_csv('/mnt/user-data/uploads/airbnb_hw.csv')
print(f"Dimensions: {airbnb.shape}")
print(f"Variables: {airbnb.columns.tolist()}")
airbnb.head()

In [ ]:
# 3.3: Cross tabulation
cross_tab = pd.crosstab(airbnb['Property Type'], airbnb['Room Type'])
cross_tab_pct = pd.crosstab(airbnb['Property Type'], airbnb['Room Type'], normalize='index') * 100

print("Counts:")
print(cross_tab)
print("\nProportions (%):")
print(cross_tab_pct.round(1))

cross_tab_pct.plot(kind='bar', stacked=True, figsize=(12, 5), edgecolor='black')
plt.title('Room Type Distribution by Property Type')
plt.ylabel('Percentage')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nPattern: Private rooms more common in houses, condos, townhouses vs apartments.")

In [ ]:
# 3.4: Price analysis
print("Original Price:")
print(airbnb['Price'].describe())

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

axes[0, 0].hist(airbnb['Price'], bins=50, edgecolor='black')
axes[0, 0].set_title('Histogram')

airbnb['Price'].plot(kind='kde', ax=axes[0, 1], linewidth=2)
axes[0, 1].set_title('KDE')

axes[1, 0].boxplot(airbnb['Price'])
axes[1, 0].set_title('Box Plot')
axes[1, 0].set_ylim(0, 500)

axes[1, 1].text(0.1, 0.5, airbnb['Price'].describe().to_string(), 
                fontsize=10, family='monospace', verticalalignment='center')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\nData badly scaled: YES. Many outliers: YES.")

In [ ]:
# Log transformation
airbnb['price_log'] = np.log(airbnb['Price'])

print("Log(Price):")
print(airbnb['price_log'].describe())

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

axes[0, 0].hist(airbnb['price_log'], bins=50, edgecolor='black', color='green')
axes[0, 0].set_title('Histogram')

airbnb['price_log'].plot(kind='kde', ax=axes[0, 1], linewidth=2, color='green')
axes[0, 1].set_title('KDE')

axes[1, 0].boxplot(airbnb['price_log'])
axes[1, 0].set_title('Box Plot')

axes[1, 1].text(0.1, 0.5, airbnb['price_log'].describe().to_string(), 
                fontsize=10, family='monospace', verticalalignment='center')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\nLog transformation: Much more symmetric, fewer extreme outliers.")

In [ ]:
# 3.5: Scatterplot and groupby
plt.figure(figsize=(12, 5))
plt.scatter(airbnb['Beds'], airbnb['price_log'], alpha=0.3, s=10)
plt.xlabel('Beds')
plt.ylabel('Log(Price)')
plt.title('Log(Price) vs Beds')
plt.grid(alpha=0.3)
plt.show()

print("\nPrice by number of Beds:")
print(airbnb.groupby('Beds')['Price'].describe())

print("\nPattern: Positive relationship. Mean and std both increase with beds.")

In [ ]:
# 3.6: Color by Room Type and Property Type
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# By Room Type
for room_type in airbnb['Room Type'].unique():
    subset = airbnb[airbnb['Room Type'] == room_type]
    axes[0].scatter(subset['Beds'], subset['price_log'], alpha=0.3, s=10, label=room_type)
axes[0].set_xlabel('Beds')
axes[0].set_ylabel('Log(Price)')
axes[0].set_title('Colored by Room Type')
axes[0].legend()
axes[0].grid(alpha=0.3)

# By Property Type (top 5)
top_props = airbnb['Property Type'].value_counts().head(5).index
for prop in top_props:
    subset = airbnb[airbnb['Property Type'] == prop]
    axes[1].scatter(subset['Beds'], subset['price_log'], alpha=0.3, s=10, label=prop)
axes[1].set_xlabel('Beds')
axes[1].set_ylabel('Log(Price)')
axes[1].set_title('Colored by Property Type (Top 5)')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nBy Room Type:")
print(airbnb.groupby('Room Type')['Price'].describe()[['mean', 'std']])

print("\nBy Property Type (Top 10):")
print(airbnb.groupby('Property Type')['Price'].describe()[['mean', 'std']].sort_values('mean', ascending=False).head(10))

print("\nMedian more reliable than mean due to right skew and outliers.")

In [ ]:
# 3.7: Jointplot with hexbin
g = sns.jointplot(data=airbnb, x='Beds', y='price_log', kind='hex', height=8, gridsize=30)
g.set_axis_labels('Number of Beds', 'Log(Price)')
plt.tight_layout()
plt.show()

print("\nHexbin reveals: Vast majority of data at 1-2 beds. High bed counts are sparse.")
print("Effect: Patterns at 5+ beds less reliable due to small sample size.")
print(f"% with ≤2 beds: {(airbnb['Beds'] <= 2).sum() / len(airbnb) * 100:.1f}%")

## Q4: Drilling Rigs Time Series

In [ ]:
# 4.1: Load and examine
rigs = pd.read_csv('/mnt/user-data/uploads/drilling_rigs.csv')
print(f"Observations: {rigs.shape[0]}, Variables: {rigs.shape[1]}")
print("\nFirst rows:")
print(rigs.head())
print("\nData types:")
print(rigs.dtypes)

print("\nCleaning needed:")
print("1. Month column: Convert to datetime")
print("2. Numeric columns: 'Not Available' values need conversion to NaN")
print("3. Cast numeric columns from object to float/int")

In [ ]:
# 4.2: Convert Month to datetime
rigs['time'] = pd.to_datetime(rigs['Month'], format='mixed')
print(f"Date range: {rigs['time'].min()} to {rigs['time'].max()}")

In [ ]:
# 4.3: Line plot of Active Well Service Rig Count
rigs['Active_Rigs'] = pd.to_numeric(rigs['Active Well Service Rig Count (Number of Rigs)'], errors='coerce')

plt.figure(figsize=(14, 5))
plt.plot(rigs['time'], rigs['Active_Rigs'], linewidth=1.5)
plt.xlabel('Year')
plt.ylabel('Active Rigs')
plt.title('Active Well Service Rig Count Over Time')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Sharp increase 1970s to early 1980s peak")
print("- Dramatic decline through 1980s")
print("- Relatively stable at lower levels since ~1990")
print(f"Peak: {rigs['Active_Rigs'].max():.0f}, Minimum: {rigs['Active_Rigs'].min():.0f}")

In [ ]:
# 4.4: First difference
rigs['Active_Rigs_Diff'] = rigs['Active_Rigs'].diff()

plt.figure(figsize=(14, 5))
plt.plot(rigs['time'], rigs['Active_Rigs_Diff'], linewidth=1, alpha=0.7)
plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.xlabel('Year')
plt.ylabel('Month-to-Month Change')
plt.title('First Difference: Monthly Change in Active Rigs')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Extreme volatility in 1970s-1980s")
print("- Much smaller fluctuations in recent decades")
print("- Oscillates around zero (stationary)")
print(f"Largest increase: {rigs['Active_Rigs_Diff'].max():.0f}")
print(f"Largest decrease: {rigs['Active_Rigs_Diff'].min():.0f}")

In [ ]:
# 4.5: Melt and plot onshore vs offshore
rigs['Onshore'] = pd.to_numeric(rigs['Crude Oil and Natural Gas Rotary Rigs in Operation, Onshore (Number of Rigs)'], errors='coerce')
rigs['Offshore'] = pd.to_numeric(rigs['Crude Oil and Natural Gas Rotary Rigs in Operation, Offshore (Number of Rigs)'], errors='coerce')

# Melt
melted = rigs[['time', 'Onshore', 'Offshore']].melt(id_vars='time', var_name='Location', value_name='Rig_Count')

# Plot
plt.figure(figsize=(14, 5))
for location in ['Onshore', 'Offshore']:
    data = melted[melted['Location'] == location]
    plt.plot(data['time'], data['Rig_Count'], linewidth=2, label=location)

plt.xlabel('Year')
plt.ylabel('Number of Rigs')
plt.title('Onshore vs Offshore Rotary Rigs')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nComparison:")
print(f"Onshore max: {rigs['Onshore'].max():.0f}")
print(f"Offshore max: {rigs['Offshore'].max():.0f}")
print("Both show boom-bust cycles, but onshore declined more dramatically after 1980s.")